In [1]:
import pandas as pd
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

In [2]:
train_df = pd.read_csv("Train_data.csv")
test_df = pd.read_csv("Test_data.csv")

In [3]:
train_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71


In [5]:
train_numeric_data = train_df.select_dtypes(include=np.number)
train_numeric_data["is_attack"] = train_df["class"].apply(lambda x: 1 if x != 'normal' else 0)
train_numeric_data.astype(np.number)
train_numeric_data.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,is_attack
0,0,491,0,0,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,0,146,0,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,0,0,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,0,232,8153,0,0,0,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,0,199,420,0,0,0,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


In [6]:
train_numeric_data['is_attack'].value_counts()

0    13449
1    11743
Name: is_attack, dtype: int64

In [7]:
x = train_numeric_data.drop(['is_attack'], axis=1)
y = train_numeric_data['is_attack']

Modelo K Neighbors

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=75)

model = KNeighborsClassifier()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

matrix = confusion_matrix(Y_test, Y_pred)

print(matrix)

print("Precision:", precision_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print("Recall: ", recall_score(Y_test, Y_pred))
print("F1 Score: ", f1_score(Y_test, Y_pred))

[[3326   51]
 [  51 2870]]
Precision: 0.9825402259500171
Accuracy: 0.9838043823436011
Recall:  0.9825402259500171
F1 Score:  0.9825402259500171


In [9]:
test_numeric_data = test_df.select_dtypes(include=np.number)
y_pred = model.predict(test_numeric_data)
array = np.asarray([y_pred])
pd.DataFrame(array).to_csv('kneighbors.csv', header=False)

Modelo PCA

In [10]:
pca = PCA(n_components=2, random_state=75)
pca_X = pca.fit_transform(x)

X_train, X_test, Y_train, Y_test = train_test_split(pca_X, y, test_size=0.25, random_state=42)

model = KNeighborsClassifier()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

matrix = confusion_matrix(Y_test, Y_pred)

print(matrix)

print("Precision:", precision_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print("Recall: ", recall_score(Y_test, Y_pred))
print("F1 Score: ", f1_score(Y_test, Y_pred))

[[3285   43]
 [  35 2935]]
Precision: 0.9855607790463399
Accuracy: 0.9876151159098127
Recall:  0.9882154882154882
F1 Score:  0.9868863483523873


In [11]:
pca = PCA(n_components=4, random_state=75)
pca_X = pca.fit_transform(x)

X_train, X_test, Y_train, Y_test = train_test_split(pca_X, y, test_size=0.25, random_state=42)

model = KNeighborsClassifier()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

matrix = confusion_matrix(Y_test, Y_pred)

print(matrix)

print("Precision:", precision_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print("Recall: ", recall_score(Y_test, Y_pred))
print("F1 Score: ", f1_score(Y_test, Y_pred))

[[3276   52]
 [  49 2921]]
Precision: 0.9825092499159098
Accuracy: 0.98396316290886
Recall:  0.9835016835016835
F1 Score:  0.9830052162207641


In [12]:
pca = PCA(n_components=6, random_state=75)
pca_X = pca.fit_transform(x)

X_train, X_test, Y_train, Y_test = train_test_split(pca_X, y, test_size=0.25, random_state=42)

model = KNeighborsClassifier()
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

matrix = confusion_matrix(Y_test, Y_pred)

print(matrix)

print("Precision:", precision_score(Y_test, Y_pred))
print('Accuracy:', accuracy_score(Y_test, Y_pred))
print("Recall: ", recall_score(Y_test, Y_pred))
print("F1 Score: ", f1_score(Y_test, Y_pred))

[[3287   41]
 [  48 2922]]
Precision: 0.9861626729665879
Accuracy: 0.9858685296919657
Recall:  0.9838383838383838
F1 Score:  0.9849991572560256


In [13]:
pca_test_X = pca.fit_transform(test_numeric_data)
pca_pred_Y = model.predict(pca_test_X)
array = np.asarray([pca_pred_Y])
pd.DataFrame(array).to_csv('pca.csv', header=False)

Como se puede observar en los resultados de las metricas obtenidas mediante PCA, el valor de N = 2 fue el que dio mejores resultados, dando un F1 Score de 0.987. Igualmente, si comparamos el clasificador del modelo de K Neighbors, este dio un resultado de F1 Score de 0.982. Por lo tanto, se concluye que PCA con N = 2 fue el modelo que dio resultados mas optimos.